In [6]:
!pip install datasets

In [7]:
import os
import time
import uuid
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from datasets import load_dataset

# Load environment variables
load_dotenv()

# Set up Pinecone API key and index name
api_key = os.getenv("PINECONE_API_KEY")
index_name = "l6v2"
custom_namespace = 'chatbot-legal'

# Initialize Pinecone with the API key
pc = Pinecone(api_key=api_key)
index = pc.Index(index_name)

# Check initial index stats
print("BEFORE:", index.describe_index_stats())

# Initialize the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load the dataset from Hugging Face
dataset = load_dataset("nisaar/Lawyer_GPT_India", split="train")

# Iterate through each row in the dataset
for row in dataset:
    question = row["question"]  # Question
    answer = row["answer"]  # Answer

    # Generate embeddings for both the question and answer
    embeddings = model.encode([question, answer])

    # Upsert the embeddings into Pinecone with unique IDs and metadata
    index.upsert(
        vectors=[
            # Embedding for the question (question)
            {
                "id": str(uuid.uuid4()),
                "values": embeddings[0].tolist(),
                "metadata": {
                    "question": question,
                    "answer": answer,
                    "created_at": int(time.time())
                },
            },
            # Embedding for the answer (answer)
            {
                "id": str(uuid.uuid4()),
                "values": embeddings[1].tolist(),
                "metadata": {
                    "question": question,
                    "answer": answer,
                    "created_at": int(time.time())
                },
            },
        ],
        namespace=custom_namespace,
    )

# Check index stats after upserting
print("Record count AFTER adding knowledge ->", index.describe_index_stats())


BEFORE: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Sethi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sethi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

c:\Users\Sethi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sethi\.cache\huggingface\hub\datasets--nisaar--Lawyer_GPT_India. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


150_lawergpt_dataset_qna_v1_train.jsonl:   0%|          | 0.00/72.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

Record count AFTER adding knowledge -> {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'chatbot-legal': {'vector_count': 424}},
 'total_vector_count': 424}
